In [1]:
%load_ext autoreload
%autoreload 2

import rlpg.envs.cars as cars
import rlpg.trainers.vae_trainer as vae_trainer
import rlpg.trainers.ppo_trainer as ppo
import rlpg.models.actor_critic as ac
import rlpg.envs.types as rl_envs

import matplotlib.pyplot as plt
import torch.utils.tensorboard
import torchvision.transforms.functional as ttf
import lightning_logs
import gymnasium as gym

In [5]:
vae_model = vae_trainer.VAEModule2D.load_from_checkpoint("/Users/danem/Programming/python/rl-pg2/checkpoints/car_vae_50_50x50_best.ckpt")
vae_model.params.device = "cpu"
ppo_params = ppo.PPOHParams(
    device = vae_model.params.device
)

env, evaluator = cars.car_vae_observer(vae_model.model.encoder, hparams = vae_model.params, items=3)

actor = ac.ActorModelContinuous(vae_model.latent_dims, env.action_space.shape[0])
critic = ac.CriticModel(vae_model.latent_dims, env.action_space.shape[0])

tstate = ppo.make_train_state_continuous(
    env = env,
    evaluator = evaluator,
    actor_model = actor,
    critic_model = critic,
    observation_size = [3,vae_model.latent_dims],
    action_space = env.action_space.shape[0],
    hparams = ppo_params,
    log_dir = "./logs"
)

# ppo.train(tstate, 10_000)
ppo.generate_episode(tstate)


/Users/danem/Programming/python/rl-pg2/rlpg/trainers/ppo_trainer.py:209: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state).to(tstate.device)


(54.61895043731668, 998)

In [ ]:
env, evaluator = cars.car_vae_observer(encoder, hparams = vae_model.params)
env = cars.AgressiveDriving(env, 3)
a, b = env.reset()
for i in range(50):
    obs, _, _, _, _ = env.step(env.action_space.sample())

p = vae_model.model.decoder(obs).view(3,50,50)
rl_envs.apply_renderopts(p, rl_envs.RenderOptions((100,100)))


In [ ]:
images = []
rewards = []
crit_vals = []
states = []
env, _ = cars.car_vae_observer(vae_model.model.encoder, vae_model.params)
state, _ = env.reset()
for i in range(1000):
    states.append(state)
    img = vae_model.model.decoder(state).view(1,50,75)
    images.append(ttf.to_pil_image(img))
    action, _ = actor.sample(actor(state))
    res = env.step(action.squeeze().tolist())
    rewards.append(res.reward)
    crit_vals.append(critic(state, action).item())
    if res.terminated or res.truncated:
        break

def boo (pair):
    fig, axs = plt.subplots(1,2)
    axs[0].imshow(pair[0])
    axs[0].set_title(f"true reward: {pair[1]}, estimate: {pair[2]}")

    dist = actor.distribution(actor(pair[3]))
    vs = dist.sample((3000,)).squeeze(1)
    axs[1].set_xlim([-3, 3])
    axs[1].set_ylim([-3, 3])
    axs[1].plot(vs[:,0], vs[:,1], 'o')

print(np.sum(rewards))
utils.slider(boo)(list(zip(images, rewards, crit_vals, states)))

In [26]:
import functools
env = gym.make_vec("CarRacing-v2", num_envs=3,)
# env = gym.make("CarRacing-v2")
env = cars.AgressiveDriving(env, 4)
env.reset()
for i in range(300):
    obs, reward, term, trunc, info = env.step(env.action_space.sample())
    if term.any():
        break

In [30]:
import rlpg.rl_memory as memomry
import tqdm

buf = torch.zeros((10_000,)).to("mps")
mem = memomry.CircularBuffer(10_000)
for x in tqdm.tqdm(range(10)):
    for i in range(10_000):
        idx = mem.get_push_idx()
        buf[i] = i

buf = []
for x in tqdm.tqdm(range(10)):
    for i in range(20_000):
        buf.append(i)
    buf.clear()



  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 657.85it/s]


In [7]:
tstate.memory.states[0].shape

torch.Size([3, 50])